<a href="https://colab.research.google.com/github/JonAyoG/3DGS/blob/3dgs_baseline/3dgs_baseline_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing 3dgs from github

In [2]:
%cd /content
!rm -rf gaussian-splatting
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile


/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 603, done.
remote: Total 603 (delta 0), reused 0 (delta 0), pack-reused 603 (from 1)
Receiving objects: 100% (603/603), 2.09 MiB | 53.59 MiB/s, done.
Resolving deltas: 100% (344/344), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3293, done.        
remote: Counting objects: 100% (322/322), done.        
remote: Compressing objects: 100% (174/174), done.        
remote: Total 3293 (delta 171), reused 280 (delta 148), pack-reused 2971 (from 1)        
Receiving objects:

In [3]:
%cd /content/gaussian-splatting/submodules/simple-knn

# Add #include <float.h> to the top of the CUDA source file
!sed -i '1i #include <float.h>' simple_knn.cu  || sed -i '1i #include <float.h>' simple-knn.cu

# Install in editable mode
!pip install -q -e .


/content/gaussian-splatting/submodules/simple-knn
  Preparing metadata (setup.py) ... done


In [4]:
%cd /content/gaussian-splatting
!pip install -q -e submodules/diff-gaussian-rasterization


/content/gaussian-splatting
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup for dataloading

In [5]:
# ===================== CONFIG =====================

# Where your training data should come from:
#   "drive_zip"  -> your own COLMAP-processed scene zipped on Drive
#   "tandt"      -> Tanks & Temples demo scene from camenduru / HuggingFace
DATA_SOURCE = "drive_zip"   # or "tandt"

# Short name for this experiment / scene
SCENE_NAME = "myshelf"      # e.g. "myshelf", "tandt_train", "room1"

# Where to store trained runs on Drive
RUNS_ROOT = "/content/drive/MyDrive/ADL4CV/3DGSruns"

# Training iterations
N_ITER = 30000               # 7000 for quick preview, 30000 for full baseline

# ------------- only used when DATA_SOURCE == "drive_zip" -------------

# Zip file of your 3DGS-ready dataset (images/ + sparse/0/) on Drive
# Example: a zip that contains a folder "myshelf_scene_3dgs/" with images/ and sparse/0/ inside.
DATA_ZIP_PATH = "/content/drive/MyDrive/ADL4CV/3DGSruns/myshelf_scene_3dgs.zip"

# Folder name *inside* that zip (the dataset root)
DATASET_SUBDIR_DRIVE = "myshelf_scene_3dgs"

# ------------- only used when DATA_SOURCE == "tandt" -----------------

# For T&T, after unzipping tandt_db.zip, training data is under tandt/train
DATASET_SUBDIR_TANDT = "tandt/train"

# ===================================================


In [6]:
import os

%cd /content/gaussian-splatting

if DATA_SOURCE == "drive_zip":
    print("📦 Using dataset from Drive zip:", DATA_ZIP_PATH)

    # Unzip your own scene into the repo
    if not os.path.exists(DATA_ZIP_PATH):
        raise FileNotFoundError(f"DATA_ZIP_PATH not found: {DATA_ZIP_PATH}")

    !unzip -q "{DATA_ZIP_PATH}" -d /content/gaussian-splatting

    DATASET_DIR = f"/content/gaussian-splatting/{DATASET_SUBDIR_DRIVE}"

elif DATA_SOURCE == "tandt":
    print("🌐 Using Tanks & Temples dataset from HuggingFace")

    # Download & unzip only if not already present
    if not os.path.exists("/content/gaussian-splatting/tandt"):
        !wget -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
        !unzip -q tandt_db.zip

    DATASET_DIR = f"/content/gaussian-splatting/{DATASET_SUBDIR_TANDT}"

else:
    raise ValueError(f"Unknown DATA_SOURCE: {DATA_SOURCE}")

print("✅ DATASET_DIR set to:", DATASET_DIR)

# Quick sanity checks
!ls "{DATASET_DIR}"
!ls "{DATASET_DIR}/images" | head
!ls "{DATASET_DIR}/sparse"


/content/gaussian-splatting
📦 Using dataset from Drive zip: /content/drive/MyDrive/ADL4CV/3DGSruns/myshelf_scene_3dgs.zip
✅ DATASET_DIR set to: /content/gaussian-splatting/myshelf_scene_3dgs
images	sparse
frame_00001.jpg
frame_00002.jpg
frame_00003.jpg
frame_00004.jpg
frame_00005.jpg
frame_00006.jpg
frame_00007.jpg
frame_00008.jpg
frame_00009.jpg
frame_00010.jpg
0


 ## Training

In [7]:
%cd /content/gaussian-splatting
!python train.py -s "{DATASET_DIR}" --iterations {N_ITER}


/content/gaussian-splatting
2025-11-09 17:10:34.129910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762708234.149415    8674 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762708234.155375    8674 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762708234.171283    8674 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762708234.171309    8674 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762708234.171312    8674 computation_placer.cc

## Save code

In [8]:
import os, glob

%cd /content/gaussian-splatting

# Find the most recently modified run under output/
runs = sorted(glob.glob("output/*"), key=os.path.getmtime)
if not runs:
    raise RuntimeError("No runs found in output/. Did training fail?")
MODEL_DIR = os.path.join("/content/gaussian-splatting", runs[-1])
RUN_ID = os.path.basename(MODEL_DIR)

print("Using MODEL_DIR:", MODEL_DIR)
print("RUN_ID:", RUN_ID)

# Optional: sync to Drive as persistent baseline
DRIVE_RUN_DIR = f"{RUNS_ROOT}/{SCENE_NAME}/{RUN_ID}"
print("Saving run to Drive at:", DRIVE_RUN_DIR)

!mkdir -p "$DRIVE_RUN_DIR"
!rsync -ah --info=progress2 "$MODEL_DIR/" "$DRIVE_RUN_DIR/"

# Quick check that the point cloud exists
!find "$MODEL_DIR" -maxdepth 5 -type f -name "point_cloud.ply" -print


/content/gaussian-splatting
Using MODEL_DIR: /content/gaussian-splatting/output/8eb1719f-8
RUN_ID: 8eb1719f-8
Saving run to Drive at: /content/drive/MyDrive/ADL4CV/3DGSruns/myshelf/8eb1719f-8
        544.59M 100%  275.51MB/s    0:00:01 (xfr#6, to-chk=0/10)
/content/gaussian-splatting/output/8eb1719f-8/point_cloud/iteration_7000/point_cloud.ply
/content/gaussian-splatting/output/8eb1719f-8/point_cloud/iteration_30000/point_cloud.ply


## Render

In [9]:
# Choose whether to render from local or Drive copy
MODEL_TO_RENDER = MODEL_DIR  # or DRIVE_RUN_DIR

%cd /content/gaussian-splatting
!python render.py -m "$MODEL_TO_RENDER" -s "$DATASET_DIR"


/content/gaussian-splatting
Looking for config file in /content/gaussian-splatting/output/8eb1719f-8/cfg_args
Config file found: /content/gaussian-splatting/output/8eb1719f-8/cfg_args
Rendering /content/gaussian-splatting/output/8eb1719f-8
Loading trained model at iteration 30000 [09/11 19:16:39]
Reading camera 227/227 [09/11 19:16:40]
Loading Training Cameras [09/11 19:16:40]
Loading Test Cameras [09/11 19:16:52]
Rendering progress: 100% 227/227 [05:24<00:00,  1.43s/it]
Rendering progress: 0it [00:00, ?it/s]


In [10]:
import glob, os

renders_dirs = glob.glob(f"{MODEL_TO_RENDER}/train/ours_*/renders")
renders_dirs += glob.glob(f"{MODEL_TO_RENDER}/test/ours_*/renders")
renders_dirs = sorted(renders_dirs)

if not renders_dirs:
    raise RuntimeError("No renders/ dirs found. Did render.py succeed?")

for d in renders_dirs:
    print("Found renders at:", d)

RENDERS_DIR = renders_dirs[0]  # pick the first (usually train/ours_<iter>/renders)
print("Using RENDERS_DIR:", RENDERS_DIR)


Found renders at: /content/gaussian-splatting/output/8eb1719f-8/test/ours_30000/renders
Found renders at: /content/gaussian-splatting/output/8eb1719f-8/train/ours_30000/renders
Using RENDERS_DIR: /content/gaussian-splatting/output/8eb1719f-8/test/ours_30000/renders


## Video (Optional)

In [ ]:
# OUTPUT_MP4 = f"/content/{SCENE_NAME}_{RUN_ID}.mp4"
# print("Writing video to:", OUTPUT_MP4)

# !ffmpeg -y -framerate 30 -i "$RENDERS_DIR/%05d.png" \
#   -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" \
#   -c:v libx264 -pix_fmt yuv420p "$OUTPUT_MP4"

# # Copy video next to the run on Drive for safekeeping
# !cp "$OUTPUT_MP4" "$DRIVE_RUN_DIR/{SCENE_NAME}_{RUN_ID}.mp4"
# print("Copied video to:", f"{DRIVE_RUN_DIR}/{SCENE_NAME}_{RUN_ID}.mp4")


## Metrics
* NOTE: only works on datasets with test data





In [11]:
%cd /content/gaussian-splatting
!python metrics.py -m "$MODEL_TO_RENDER"


/content/gaussian-splatting

Scene: /content/gaussian-splatting/output/8eb1719f-8
Method: ours_30000
Metric evaluation progress: 0it [00:00, ?it/s]
  SSIM :          nan
  PSNR :          nan
  LPIPS:          nan



In [12]:
%cd /content/gaussian-splatting
!python metrics.py -h


/content/gaussian-splatting
usage: metrics.py [-h] --model_paths MODEL_PATHS [MODEL_PATHS ...]

Training script parameters

options:
  -h, --help            show this help message and exit
  --model_paths MODEL_PATHS [MODEL_PATHS ...], -m MODEL_PATHS [MODEL_PATHS ...]
